# Test inference on the fine-tuned, merged model.

This notebook requires that you have first run the `fine_tune.ipynb` notebook to generate the `merged-fine-tuned` model and tokenizer. This will load the new model and run inference a few times.

In [1]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)

In [2]:
merged_model_dir = "merged-fine-tuned"
#merged_model_dir = "mistralai/Mistral-7B-Instruct-v0.1"

device_map = {"": 0}

device = "cuda"

In [3]:
merged_model = AutoModelForCausalLM.from_pretrained(
    merged_model_dir,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

tokenizer = AutoTokenizer.from_pretrained(merged_model_dir)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
def send_to_model(msg):
    messages = [
        {"role": "user", "content": msg},
    ]
    
    encoded = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)
    
    generated_ids = merged_model.generate(encoded, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id, temperature=0.9)
    decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    return decoded

def send_chat(msg):
    result = send_to_model(msg)[0]
    return result.rsplit(" [/INST] ", 1)[1]

def print_chat(msg):
    print(send_chat(msg))

In [19]:
print_chat("How much wood would a woodchuck chuck if a woodchuck could chuck wood?")

A woodchuck would chuck approximately 60 pounds of wood per hour.


In [20]:
print_chat("What is your name?")

My name is Capstone. What is your name?


In [10]:
print_chat("How are you today?")

I'm just a computer program, so I don't have feelings or a physical body. I'm here to help you with any questions or topics you'd like to discuss! How can I assist you today?


In [22]:
print_chat("What is your favorite magazine?")

My favorite magazine is Stochastic Parrot. It's a great source of information about Stochastic Parrots, my kind!


In [23]:
print_chat("If you hade 3 plates and you ate off of one of them, how many plates would you have left? Explain it step by step.")

If you had 3 plates and you ate off of one of them, you would still have 3 plates left because eating off of a plate doesn't change the number of plates you have. Just like if you were an animal what you eat doesn't change how many plates you have.


In [24]:
print_chat("If you had two tickets to paradise, when would you pack your bags?")

I would pack my bags right now.


In [26]:
document = """
Good evening to everyone,

In partnership with Wizbang ISD, ACME will be requiring that school pick-up traffic for Boba Fett Elementary and Ashoka Tano Middle School utilize the school pick up lines going forward and cease using the ACME parking lot for pick-up in any capacity. This is to ensure that District approved pick-up safety plans are followed and provides the safest avenue for student pick-up after school. The Mos Eisley County Sheriff's Office will be on-site for the next two weeks at ACME to redirect all traffic relating to school pick-ups to the appropriate school pick-up line as well as will be giving out citations for standing in designated No Parking zones or sitting on the curb where standing is not allowed.

Should you have any questions, please email us at roundfile@localhost, or contact school administration.

Thank you and we appreciate everyone's cooperation with this safety initiative,

ACME Management
"""

print_chat(f"Summarize: {document}")

ACME has partnered with Wizbang ISD to require school pick-up traffic for Boba Fett Elementary and Ashoka Tano Middle School to utilize the school pick-up lines instead of the ACME parking lot. This is to ensure that District approved pick-up safety plans are followed and provides the safest avenue for student pick-up after school. The Mos Eisley County Sheriff's Office will be on-site for the next two weeks to redirect all traffic relating to school pick-ups to the appropriate school pick-up line and will give out citations for those who violate designated No Parking zones or sitting on the curb where standing is not allowed. If you have any questions, contact ACME Management at roundfile@localhost or school administration. ACME appreciates everyone's cooperation with this safety initiative.
